The goal of this notebook is to obtain the recalibrated intercepts for the WMH severity ordered logistic model. Parts of this notebook need to be run 3 times, once for each intercept, and the intermediate results obtained need to be added to the WMHSeverityModelRecalibrated class.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import importlib.util

from microsim.population_factory import PopulationFactory
from microsim.outcome import OutcomeType
from microsim.wmh_severity import WMHSeverityModel, WMHSeverity

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [2]:
pd.set_option('future.no_silent_downcasting', True) 

In [3]:
popSize = 500000
pop = PopulationFactory.get_kaiser_population(n=popSize)
pop.advance(1, nWorkers=5)

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population_factory.py:525: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,dfForGroup])
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [16]:
class WMHSeverityModelRecalibrated(WMHSeverityModel):
    def __init__(self):
        pass
    def estimate_next_risk(self, person, interceptChange=0.):
        lpWithoutIntercept = self.calc_linear_predictor_for_patient_characteristics(
            person._gender,
            person._raceEthnicity,
            person._smokingStatus,
            person._statin[-1],
            person._afib[-1],
            person._pvd[-1],
            person._age[-1],
            person._sbp[-1],
            person._dbp[-1],
            person._bmi[-1],
            person._anyPhysicalActivity[-1],
            person._antiHypertensiveCount[-1],
            #person._otherLipidLowering,
            person._a1c[-1],
            person._totChol[-1],
            person._hdl[-1],
            person._ldl[-1],
            person._trig[-1],
            person._creatinine[-1],
            person._modality)
        
        #obtain the linear predictors
        #lpNoWMH = lpWithoutIntercept + 8.2116 + interceptChange #start with this
        lpNoWMH = lpWithoutIntercept + 8.2116 + -0.26733 #this will be used when the recalibration is done for this intercept
        #lpMildWMH = lpWithoutIntercept + interceptChange #in the 2nd iteration use this
        lpMildWMH = lpWithoutIntercept + 10.2237 + -0.43271 #and when the 2nd iteration is done use the optimized intercept
        lpModerateWMH = lpWithoutIntercept + 11.6124 + interceptChange #in the 3rd iteration use this
        #obtain the first three cumulative probabilities for the first three classes, last cumulative probability is 1.
        noWMHCumulative = self.inverse_logit(lpNoWMH)
        mildWMHCumulative = self.inverse_logit(lpMildWMH)
        moderateWMHCumulative = self.inverse_logit(lpModerateWMH)
        #make the decision
        draw = person._rng.uniform()
        if draw<noWMHCumulative:
            return WMHSeverity.NO
        elif draw<mildWMHCumulative:
            return WMHSeverity.MILD
        elif draw<moderateWMHCumulative:
            return WMHSeverity.MODERATE
        elif draw<1.:
            return WMHSeverity.SEVERE
        else:
            raise RuntimeError("Draw inconsistent with cumulative probabilities in WMHSeverityModel.")

In [17]:
kaiserUnknownRate = 0.069
kaiserNoRate = 0.7065/(1-kaiserUnknownRate)
kaiserMildRate = 0.1715/(1-kaiserUnknownRate)
kaiserModerateRate = 0.0378/(1-kaiserUnknownRate)
#what rate I am recalibrating, start with No in 1st iteration, then Mild in 2nd, Moderate in 3rd
kaiserRate = kaiserModerateRate

In [12]:
def get_rate(people, severity, interceptChange=0):
    return np.mean(list(map(lambda x: WMHSeverityModelRecalibrated().estimate_next_risk(x, interceptChange=interceptChange).value==severity, 
                             list(filter(lambda x: x._outcomes[OutcomeType.WMH][0][1].wmhSeverityUnknown==False, pop._people)))))

In [18]:
interceptChange=0
interceptChangeStep = 0.5
simNoRate = get_rate(pop._people, 'no', 0)
simMildRate = get_rate(pop._people, 'mild', 0)
simModerateRate = get_rate(pop._people, 'moderate', 0)
#what rate I am recalibrating, start with No in 1st iteration, then Mild in 2nd, Moderate in 3rd
simRate = simModerateRate
diff = kaiserRate-simRate
if diff>0:
    interceptChange += interceptChangeStep
else:
    interceptChange += -interceptChangeStep
print(kaiserRate, simRate, diff, interceptChange, interceptChangeStep)

0.04060150375939849 0.046875336732626754 -0.006273832973228262 -0.5 0.5


In [19]:
for i in range(30):
    #modify severity with each iteration
    simRate =  get_rate(pop._people, 'moderate', interceptChange=interceptChange)
    print(kaiserRate, simRate, diff, interceptChange, interceptChangeStep)    
    diffOld = diff
    diff = kaiserRate-simRate
    if diffOld*diff<0:
        interceptChangeStep = interceptChangeStep/2.
    if diff>0:
        interceptChange += interceptChangeStep
    else:
        interceptChange += -interceptChangeStep

0.04060150375939849 0.040961773034666756 -0.006273832973228262 -0.5 0.5
0.04060150375939849 0.030194949333862048 -0.0003602692752682629 -1.0 0.5
0.04060150375939849 0.035778784443706926 0.010406554425536445 -0.75 0.25
0.04060150375939849 0.04071824799900004 0.004822719315691566 -0.5 0.25
0.04060150375939849 0.0392743384954894 -0.0001167442396015489 -0.625 0.125
0.04060150375939849 0.03959113655073725 0.0013271652639090942 -0.5625 0.0625
0.04060150375939849 0.03991008969479632 0.0010103672086612392 -0.5 0.0625
0.04060150375939849 0.041325983043761234 0.0006914140646021721 -0.4375 0.0625
0.04060150375939849 0.0407311785318673 -0.0007244792843627418 -0.46875 0.03125
0.04060150375939849 0.040112668043050054 -0.00012967477246880743 -0.5 0.03125
0.04060150375939849 0.04091220599200893 0.0004888357163484389 -0.484375 0.015625
0.04060150375939849 0.040326021835359833 -0.00031070223261044083 -0.4921875 0.0078125
0.04060150375939849 0.040767815041657864 0.0002754819240386591 -0.48828125 0.003906

In [20]:
simNoRate = get_rate(pop._people, 'no', 0)
simMildRate = get_rate(pop._people, 'mild', 0)
simModerateRate = get_rate(pop._people, 'moderate', 0)
simNoRate, kaiserNoRate, simMildRate, kaiserMildRate, simModerateRate, kaiserModerateRate

(0.758300324556375,
 0.7588614393125671,
 0.1836502032248749,
 0.1842105263157895,
 0.046545608144511634,
 0.04060150375939849)